#  Jove Editor

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
  #---- Leave these definitions ON if running on laptop
  #---- Else turn OFF by putting them between ''' ... '''

  # sys.path[0:0] = ['..'] #,           '../3rdparty' ]
  print("Running on own machine.")

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://gitlab.com/anon_submitter/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.lex  import lex
from jove.yacc import yacc
from jove.Def_md2mc  import *
from jove.JoveEditor import *  
from jove.AnimateDFA import *

from jove.Def_DFA import *
from jove.Def_NFA import *
from jove.Def_PDA import *
from jove.Def_TM  import *
from jove.Def_RE2NFA import *
from jove.LangDef import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# NFA defn, AnimateNFA, re2nfa, nfa2dfa, min_dfa, iso_dfa

## First, define an NFA using Jove's Automaton Markdown

In [ ]:
nfahas01 = md2mc('''
NFA
I : 0 | 1 -> I
I : ''    -> A
A : 0     -> B
B : 1     -> C
C : 0 | 1 -> C
C : ''    -> F
''')

## Let's draw this NFA now

In [ ]:
dotObj_nfa(nfahas01)

## Now let's animate the above creation

In [ ]:
from jove.AnimateNFA import *
AnimateNFA(nfahas01)
# Due to a Colab limitation, the following incantation is needed in every cell to show the play buttons!
display(HTML('<link rel=\"stylesheet\" href=\"//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css\"/>'))

## Write an alternate NFA definition and check it matches the previous

In [ ]:
nfahas01alt = re2nfa("(0+1)*01(0+1)*")
iso_dfa(min_dfa(nfa2dfa(nfahas01)), min_dfa(nfa2dfa(nfahas01alt)))

# Sometimes, building the complement of a machine is easier

In [ ]:
dotObj_dfa(comp_dfa(min_dfa(nfa2dfa(re2nfa("(0+1)*(111)(0+1)*")))), FuseEdges=True)

# Definition of DFA union in Python

$Q = (F_1 \times Q_2) \cup (Q_1 \times F_2)$

$F = F_1\cup F_2$

$q_0 = (q_0^1, q_0^2)$

$\delta((q_1,q_2),a) = (\delta_1(q_1,a), \delta_2(q_2,a)) $

## Definition of DFA union with extra checks implemented

In [ ]:
def union_dfa(D1in, D2in):
    """In : D1in (consistent DFA)
            D2in (consistent DFA)
       Out: DFA for language union of D1in, D2in (consistent DFA). 
    """
    assert(is_consistent_dfa(D1in)), "Inconsist. DFA1 in union_dfa"
    assert(is_consistent_dfa(D2in)), "Inconsist. DFA2 in union_dfa"
    if (D1in["Sigma"] != D2in["Sigma"]):
        print("Union on DFA with different alphabets.")
        print("Making alphabets the same (taking unions).")
        Sigma = D1in["Sigma"] | D2in["Sigma"]
        D1   = copy.deepcopy(D1in)
        D2   = copy.deepcopy(D2in)
        D1["Sigma"] = Sigma
        D2["Sigma"] = Sigma
        D1 = totalize_dfa(D1)
        D2 = totalize_dfa(D2)
    else:
        D1 = totalize_dfa(D1in)
        D2 = totalize_dfa(D2in)
   
    # The states can be anything in the cartesian product
    Q     = set(product(D1["Q"], D2["Q"]))
    
    # Accept if one of the DFAs accepts
    F     = (set(product(D1["F"], D2["Q"])) | 
             set(product(D1["Q"], D2["F"])))
    
    # Start a lock-step march from the respective q0
    q0    = (D1["q0"], D2["q0"])
    
    # The transition function attempts to march both
    # DFAs in lock-step per their own transition functions
    Delta = { ((q1,q2),ch) : (q1p, q2p) 
               for q1 in D1["Q"] for q1p in D1["Q"] 
               for q2 in D2["Q"] for q2p in D2["Q"] 
               for ch in D1["Sigma"] 
               if D1["Delta"][(q1,ch)] == q1p and
                  D2["Delta"][(q2,ch)] == q2p }
                                                          
    return pruneUnreach(
        mk_dfa(Q, D1["Sigma"], Delta, q0, F))


In [ ]:
D1 = min_dfa(nfa2dfa(nfahas01))
dotObj_dfa(D1, FuseEdges=True)

In [ ]:
D2 = min_dfa(nfa2dfa(re2nfa("(0+1)*(111)(0+1)*")))
dotObj_dfa(D2, FuseEdges=True)

In [ ]:
# Union of the above DFA D1 and D2
dotObj_dfa(union_dfa(D1,D2))

# Animating the star of a language

In [ ]:
# Animate the star of a language
L1 = {'a', 'b'}
L2 = {'ab', 'bc'}
# lphi()  = {}
# lunit() = {''}

wdg.interact(lstar,
            L = {'L1':L1,
                 'L2':L2,
                 'lphi' : lphi(),
                 'lunit': lunit()},
             n = (0,7))

# Postage Stamp Problem for 3, 5, and 7 cents

In [ ]:
all357 = "(111+11111+1111111)*"
dotObj_dfa(min_dfa(nfa2dfa(re2nfa(all357))))

# Brzozowski's minimization

In [ ]:
def brz_min_dfa(D):
    return(nfa2dfa(rev_dfa(nfa2dfa(rev_dfa(D)))))

## Check if it minimizes; here is the unminimzed machine

In [ ]:
dotObj_dfa(nfa2dfa(re2nfa(all357)))

# Check if it minimizes; here is the minimized machine

In [ ]:
dotObj_dfa(brz_min_dfa(nfa2dfa(re2nfa(all357))))

# PDA Animation

In [ ]:
from jove.AnimatePDA import *

## See how the markdown of Jove allows good commenting

In [ ]:
f27sip = md2mc('''PDA 
!!---------------------------------------------------------------------------
!! This is a PDA From Sipser's book
!! This matches a's and b's ignoring c's
!! or matches a's and c's, ignoring b's in the middle
!! thus matching either a^m b^m c^n or a^m b^n c^m
!!---------------------------------------------------------------------------

!!---------------------------------------------------------------------------
!! State: in , sin ; spush -> tostates !! comment
!!---------------------------------------------------------------------------
iq1    : '' , ''  ; $     -> q2       !!  start in init state by pushing a $

q2     : a  , ''  ; a     -> q2       !!  stack a's
q2     : '' , ''  ; ''    -> q3,q5    !!  split non-det for a^m b^m c^n (q3)
                                      !!  or a^m b^n c^m (q5)

q3     : b  , a   ; ''    -> q3       !!  match b's against a's
q3     : '' , $   ; ''    -> fq4      !!  hope for acceptance when $ surfaces

fq4    : c  , ''  ; ''    -> fq4      !!  be happy so long as c's come
                                      !!  will choke and reject if anything
                                      !!  other than c's come

q5     : b  , ''  ; ''    -> q5       !!  here, we are going to punt over b's
q5     : '' , ''  ; ''    -> q6       !!  and non-det decide to honor c's matching
                                      !!  against a's

q6     : c  , a   ; ''    -> q6       !!  OK to match so long as c's keep coming
q6     : '' , $   ; ''    -> fq7      !!  when $ surfaces, be ready to accept in
                                      !!  state fq7. However, anything else coming in
                                      !!  now will foil match and cause rejection.
!!---------------------------------------------------------------------------
!! You may use the line below as an empty shell to populate for your purposes
!! Also serves as a syntax reminder for entering PDAs.
!!
!! State : i1 , si1 ; sp1 | i2 , si2 ; sp2 -> tos1, tos2 !! comment
!!
!! ..    : .. , ..  ; ..  | .. , ..  ; ..  -> ..  , ..   !!  ..
!!---------------------------------------------------------------------------
!!---------------------------------------------------------------------------
!!
!! Good commenting and software-engineering methods, good clean indentation,
!! grouping of similar states, columnar alignment, etc etc. are HUGELY
!! important in any programming endeavor -- especially while programming
!! automata. Otherwise, you can easily make a mistake in your automaton
!! code. Besides, you cannot rely upon others to find your mistakes, as
!! they will find your automaton code impossible to read!
!!
!!---------------------------------------------------------------------------

''')

In [ ]:
dotObj_pda(f27sip)

In [ ]:
from jove.JoveEditor import *
AnimatePDA(f27sip, FuseEdges=True)
display(HTML('<link rel=\"stylesheet\" href=\"//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css\"/>'))

## Showing explore_pda

In [ ]:
explore_pda("aabbcc", f27sip)

# Jove Editor is a cockpit for animating all machine types

This cockpit used to work only on one's own machine, but now works also on Colab!

### JoveEditor has two optional parameters

#### machine (default: None) - You can supply an already created jove machine (DFA, NFA, PDA, TM). The machine dictionary will be displayed under 'Edit' tab and the Animation for the machine will be displayed in the 'Animate' tab

#### examples (default: False) - If True, the 'Edit' tab will contain example markdown for each machine type. Otherwise, the placeholder text displays a reminder of the rule format

In [ ]:
JoveEditor(examples=True)

# Animate* Classes
You can animate any Jove machine (DFA, NFA, PDA, TM) using the corresponding Animate* class which will create a widget to interact with the machine.

You can enter a string for the machine to consume in the 'Input:' text box at the top of the widget. The string must be composed of symbols in the machine's alphabet which is displayed in input box by default. If the string contains unknown symbols the 'Animate' button will turn red with the message 'Invalid Input' until the string is updated.

Click the 'Animate' button when you are ready for the machine to animate using your input. This disables the input textbox and enables the play controls at the bottom of the widget. When you are done exploring the current input click the 'Change Input' button to disable the play controls and re-enable the input textbox. 

The play controls at the bottom of widget (from left to right) consist of:
* Play          : play/resume the animation
* Pause         : pause the animation
* Stop          : stop the animation, play will start from the beginning
* Loop          : loop when the end of the animation is reached and continue playing from the beginning
* Step Backward : take one 'step' backward in the animation
* Step Forward  : take one 'step' forward in the animation
* Speed         : this slider lets you increase the animation playback speed

## END of SIGCSE demos